In [6]:
import requests
from bs4 import BeautifulSoup
import csv
from urllib.parse import urljoin, urlparse

search_term = "artificial intelligence site:financialexpress.com"
google_url = f"https://www.google.com/search?q={search_term}&lr=lang_en&cr=countryIN"

# Added user-agent
headers = {"User-Agent": "Mozilla/5.0"}

try:
    resp = requests.get(google_url, headers=headers)
    resp.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)

    soup = BeautifulSoup(resp.text, "html.parser")

    # Search full HTML rather than one div
    results = soup.find_all("a")

    links = []
    for result in results:
        link = result.get("href")

        # Check for relative vs full URLs
        if link.startswith("/"):
            link = urljoin(google_url, link)

        links.append(link)

    articles = [link for link in links if "financialexpress.com" in link]

    print(f"Found {len(articles)} articles")

except requests.exceptions.RequestException as e:
    print("Error searching Google:", e)
    articles = []

scraped_articles = []

# Function to check if the URL should be skipped
def should_skip_url(url):
    # Check if the URL is in the format to be skipped
    if "business" in url :
        return False
    return True

# Function to scrape title, date, and content from an article URL
def scrape_article_info(url):
    try:
        resp = requests.get(url, headers=headers)
        resp.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)

        soup = BeautifulSoup(resp.text, "html.parser")

        # Extract title
        title_tag = soup.find("title")
        title_text = title_tag.text.strip() if title_tag else ""

        # Extract date from custom_ua_data script
        script_tag = soup.find("script", text=lambda x: "var custom_ua_data" in x)
        date_text = ""
        if script_tag:
            script_content = script_tag.text
            date_start = script_content.find('"datePublished":"') + len('"datePublished":"')
            date_end = script_content.find('","', date_start)
            date_text = script_content[date_start:date_end]

        # Extract content
        content_tag = soup.find("div", class_="entry-content")
        content_text = content_tag.text.strip() if content_tag else ""

        return title_text, date_text, content_text

    except requests.exceptions.HTTPError as e:
        print(f"Error scraping article from {url}: {e}")
        return "", "", ""
    except Exception as e:
        print(f"Error processing article from {url}: {e}")
        return "", "", ""

# Loop through each article URL and scrape information
for article_url in articles:
    # Skip the URL if it matches the format to be skipped
    if should_skip_url(article_url):
        print(f"Skipping URL: {article_url}")
        continue

    title, date, content = scrape_article_info(article_url)

    print('Title:', title)
    print('Date:', date)
    print('Content:', content[:100])  # Print the first 100 characters of the content
    print('URL:', article_url)
    print('-' * 30)

    # Save scraped data
    article = {
        "title": title,
        "date": date,
        "content": content,
        "url": article_url,
    }

    scraped_articles.append(article)

# Save articles to CSV
with open("articles.csv", "w", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Title", "Date", "Content", "URL"])

    for article in scraped_articles:
        writer.writerow([article["title"], article["date"], article["content"], article["url"]])

print("Scraping complete!")


Found 36 articles
Skipping URL: https://www.google.com/search?q=artificial+intelligence+site:financialexpress.com&lr=lang_en&cr=countryIN&sca_esv=593131414&gbv=1&sei=tdiFZczhKLrN0PEP1curkAI
Skipping URL: https://www.google.com/search?q=artificial+intelligence+site:financialexpress.com&lr=lang_en&cr=countryIN&sca_esv=593131414&tbs=lr:lang_1en,ctr:countryIN&tbm=nws&source=lnms&sa=X&ved=0ahUKEwiMkOiv2aODAxW6JjQIHdXlCiIQ_AUIBigB
Skipping URL: https://www.google.com/search?q=artificial+intelligence+site:financialexpress.com&lr=lang_en&cr=countryIN&sca_esv=593131414&tbs=lr:lang_1en,ctr:countryIN&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiMkOiv2aODAxW6JjQIHdXlCiIQ_AUIBygC
Skipping URL: https://www.google.com/search?q=artificial+intelligence+site:financialexpress.com&lr=lang_en&cr=countryIN&sca_esv=593131414&tbs=lr:lang_1en,ctr:countryIN&tbm=bks&source=lnms&sa=X&ved=0ahUKEwiMkOiv2aODAxW6JjQIHdXlCiIQ_AUICCgD
Skipping URL: https://www.google.com/url?q=https://maps.google.com/maps%3Fq%3Dartificial%2Bi

<ipython-input-6-d8227a6066a3>:61: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script_tag = soup.find("script", text=lambda x: "var custom_ua_data" in x)


Title: From AI to responsible AI - the future? - Digital Transformation News | The Financial Express
Date: 2023-12-22T08:00:00+05:30
Content: As artificial intelligence(AI) takes a wrong turn with the misuse of deepfakes and generative AI, am
URL: https://www.google.com/url?q=https://www.financialexpress.com/business/digital-transformation-from-ai-to-responsible-ai-the-futurenbsp-3344646/&sa=U&ved=2ahUKEwiMkOiv2aODAxW6JjQIHdXlCiIQvOMEegQICRAC&usg=AOvVaw0rZ31Bh84RVOlGvGBglhHA
------------------------------
Title: Humanoid robots with artificial intelligence have a long way to go - Digital Transformation News | The Financial Express
Date: 2023-12-21T18:45:00+05:30
Content: Elon Musk, Masayoshi Son and other tech leaders have bet on a future that includes human-like robots
URL: https://www.google.com/url?q=https://www.financialexpress.com/business/digital-transformation-humanoid-robots-with-artificial-intelligence-have-a-long-way-to-go-3344522/&sa=U&ved=2ahUKEwiMkOiv2aODAxW6JjQIHdXlCiIQvO